In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch
import cv2
import os
import json
import plotly.graph_objects as go

from data_process import feature_map

In [2]:
with open("../cfg/cfg.json") as f:
    cfg = json.load(f)

In [3]:
dataset_dir = cfg['dataset_dir']

image_dir = os.path.join(dataset_dir, "training", "image_2")
lidar_dir = os.path.join(dataset_dir, "training", "velodyne")
calib_dir = os.path.join(dataset_dir, "training", "calib")
label_dir = os.path.join(dataset_dir, "training", "label_2")

In [4]:
def get_lidar(idx):
    lidar_file = os.path.join(lidar_dir, f"{idx:06d}.bin")
    return np.fromfile(lidar_file, dtype=np.float32).reshape(-1, 4)

def get_image(self, idx):
    img_path = os.path.join(self.image_dir, f'{idx:06d}.png')
    img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

    return img_path, img

def get_calib(self, idx):
    calib_file = os.path.join(self.calib_dir, f'{idx:06d}.txt')
    # assert os.path.isfile(calib_file)
    return Calibration(calib_file)

CLASS_NAME_TO_ID = {
    'Pedestrian': 0,
    'Car': 1,
    'Cyclist': 2,
    'Van': 1,
    'Truck': -3,
    'Person_sitting': 0,
    'Tram': -99,
    'Misc': -99,
    'DontCare': -1
}

def get_label(idx):
    labels = []
    label_path = os.path.join(self.label_dir, f'{idx:06d}.txt')
    for line in open(label_path, 'r'):
        line = line.rstrip()
        line_parts = line.split(' ')
        obj_name = line_parts[0]  # 'Car', 'Pedestrian', ...
        cat_id = int(CLASS_NAME_TO_ID[obj_name])
        if cat_id <= -99:  # ignore Tram and Misc
            continue
        '''
        truncated = int(float(line_parts[1]))  # truncated pixel ratio [0..1]
        occluded = int(line_parts[2])  # 0=visible, 1=partly occluded, 2=fully occluded, 3=unknown
        alpha = float(line_parts[3])  # object observation angle [-pi..pi]
        # xmin, ymin, xmax, ymax
        bbox = np.array([float(line_parts[4]), float(line_parts[5]), float(line_parts[6]), float(line_parts[7])])
        '''

        # height, width, length (h, w, l)
        h, w, l = float(line_parts[8]), float(line_parts[9]), float(line_parts[10])
        # location (x,y,z) in camera coord.
        x, y, z = float(line_parts[11]), float(line_parts[12]), float(line_parts[13])
        ry = float(line_parts[14])  # yaw angle (around Y-axis in camera coordinates) [-pi..pi]

        object_label = [cat_id, x, y, z, h, w, l, ry]
        labels.append(object_label)

    if len(labels) == 0:
        labels = np.zeros((1, 8), dtype=np.float32)
        has_labels = False
    else:
        labels = np.array(labels, dtype=np.float32)
        has_labels = True

    return labels, has_labels

In [5]:
point_cloud = get_lidar(6)

In [6]:
frm = feature_map.frame(point_cloud, cfg)

In [7]:
frm.pos.shape

(109149, 4)

In [ ]:
def plot_frame(pos):
    data = [go.Scatter3d(x=pos[:, 0],
                         y=pos[:, 1],
                         z=pos[:, 2],
                         mode = 'markers',
                         marker=dict(size=1, opacity=0.5, color=pos[:, 3]))]

    fig = go.Figure(data=data)
    fig.update_layout(title="frame", template='plotly_dark', scene=dict(aspectmode='data'))
    fig.show()

In [ ]:
plot_frame(frm.pos)

In [12]:
base_map_sup, height_map_sup, density_map_sup = frm.set_bev_map_cpu()

Function 'set_bev_map_cpu' executed in 0.1595s


In [ ]:
frm.pos[1], frm.pos[2]

In [ ]:
frm.set_bev_map(cfg)

frm.base_map

In [ ]:
arr = np.array([1,2,2,3,4,4,5,6,7,7,7,7])
unique_elements, element_counts = np.unique(arr, return_counts=True)

In [ ]:
unique_elements, element_counts

In [ ]:
np.where(arr[:2] > 2)

In [ ]:
cnts, edges = np.histogram(point_cloud[:, 2], bins=np.arange(-3,1,0.05))

In [ ]:
cnts.shape, edges.shape

In [ ]:
np.where(cnts > 10)[0]

In [ ]:
point_cloud.shape

In [ ]:
plt.hist(point_cloud[:, 2], bins=np.arange(-3, 3, 0.1), log=True)

In [ ]:
from util.timing import timer_func
from time import time

In [ ]:
t0 = time()

# Voxelize the pixels
NX = NY = 1024
in_range = (np.abs(point_cloud[:, 0]) < 64) & (np.abs(point_cloud[:, 1]) < 64)
point_cloud = point_cloud[in_range]

ixs = (point_cloud[:, 0] + 64).astype(np.uint16)
iys = (point_cloud[:, 1] + 64).astype(np.uint16)


print(f"time cost: {time()-t0:.2g}s")

In [ ]:
point_cloud[:, 2].min(), point_cloud[:, 2].max()

In [ ]:
h_map = torch.full((nx, ny), )